In [254]:
import torch
import torchvision
from torch.utils.data import DataLoader 
import pylab as plt
import torch.nn as nn
import torch.nn.functional as F

In [255]:
# Download the dataset

In [332]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor(),
])

In [333]:
dataset_train = torchvision.datasets.MNIST(root = '/home/suhas/research/pytorch/data/mnist', download = True, transform = transform)
dataset_test = torchvision.datasets.MNIST(root = '/home/suhas/research/pytorch/data/mnist', download = True, train = False, transform = transform)

In [334]:
len(dataset_train), len(dataset_test)

(60000, 10000)

In [335]:
dl_train = DataLoader(dataset_train, batch_size = 300, shuffle = True)
dl_test = DataLoader(dataset_test, batch_size = 200, shuffle = True)

In [336]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


In [338]:
class mnistModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 2, kernel_size = 2, stride = 2)
        self.maxpool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv2 = nn.Conv2d(in_channels = 2, out_channels = 1, kernel_size = 2, stride = 1)
        self.maxpool2 = nn.MaxPool2d(kernel_size = 2, stride = 1)
        self.linear = nn.Linear(25, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = x.flatten(1)
        x = self.linear(x)
        x = self.relu(x)
        return x

In [339]:
# Define the model, the loss function and the optimizer
model = Net()
loss_func = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters())

In [369]:
# Define a training-validation loop
epochs = 100
val_losses = []
train_losses = []
val_accuracy = []
for epoch in range(epochs): 
    model.train()
    for xb, yb in dl_train:
        xb = xb.float()
        preds = model(xb)
        train_loss = F.nll_loss(preds, yb)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

    model.eval()
    with torch.inference_mode():
        valid_loss = 0
        for xv, yv in dl_test:
            valid_preds = model(xv.float())
            valid_loss += loss_func(valid_preds, yv)
        valid_loss = valid_loss / len(dl_test)
        accuracy = ((valid_preds.argmax(dim = 1) == yv).sum()/len(yv)).item()
        print(epoch, valid_loss.item(), train_loss.item(), accuracy)
        val_losses.append(valid_loss.item())
        train_losses.append(train_loss.item())
        val_accuracy.append(accuracy)

0 2.1926429271698 2.2029221057891846 0.6399999856948853
1 2.122760772705078 2.1155343055725098 0.675000011920929
2 2.0006821155548096 2.0369884967803955 0.6650000214576721
3 1.7833932638168335 1.817724585533142 0.7549999952316284
4 1.455471158027649 1.561803936958313 0.7900000214576721
5 1.1074920892715454 1.2010411024093628 0.7950000166893005
6 0.8571884632110596 1.0089948177337646 0.8299999833106995
7 0.7032543420791626 0.9101952314376831 0.875
8 0.6078070998191833 0.7502356767654419 0.8149999976158142
9 0.5443054437637329 0.760055661201477 0.8550000190734863
10 0.4988960921764374 0.5522511005401611 0.875
11 0.46417751908302307 0.6225759387016296 0.8999999761581421
12 0.4385671317577362 0.657427191734314 0.8799999952316284
13 0.4173664450645447 0.5139011740684509 0.8899999856948853
14 0.39942967891693115 0.5732272267341614 0.9200000166893005
15 0.38443127274513245 0.48822829127311707 0.9350000023841858
16 0.3719990849494934 0.48456665873527527 0.8899999856948853
17 0.3620283901691437

KeyboardInterrupt: 

In [378]:
import pandas as pd
import numpy as np

In [375]:
dftrain = pd.read_csv("/home/suhas/Downloads/D1_Project_train.csv")

In [379]:
len(dftrain)

90000

In [380]:
np.std(dftrain.TARGET)

0.007455566382398637

In [382]:
np.mean(dftrain.TARGET)

-1.7849832934444285e-05

In [383]:
torch.__version__

'2.2.0+cu121'